In [ ]:
import requests
import os
import json
from dotenv import load_dotenv

# Load .env file
load_dotenv()

# Load the API key
API_KEY = os.getenv("MORALIS_KEY")
if not API_KEY:
    raise ValueError("Server API Key not loaded. Check your .env file.")

# Transaction hash (with or without 0x prefix)
TRANSACTION_HASH = "0xfeda0e8f0d6e54112c28d319c0d303c065d1125c9197bd653682f5fcb0a6c81e"

# Build the URL
url = f"https://deep-index.moralis.io/api/v2.2/transaction/{TRANSACTION_HASH}/verbose?chain=eth"

headers = {
    "Accept": "application/json",
    "X-API-Key": API_KEY
}

try:
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        tx_data = response.json()
        print(json.dumps(tx_data, indent=2))
        
        # Example: Extract useful information
        print("\n--- Transaction Summary ---")
        print(f"From: {tx_data['from_address_label']} ({tx_data['from_address']})")
        print(f"To: {tx_data['to_address_label']} ({tx_data['to_address']})")
        print(f"Block: {tx_data['block_number']}")
        print(f"Status: {'Success' if tx_data['receipt_status'] == '1' else 'Failed'}")
        print(f"Gas Used: {tx_data['receipt_gas_used']}")
        print(f"Transaction Fee: {tx_data['transaction_fee']} ETH")
        
        # Extract decoded events
        if tx_data.get('logs'):
            print(f"\n--- Events ({len(tx_data['logs'])} total) ---")
            for log in tx_data['logs']:
                if log.get('decoded_event'):
                    event = log['decoded_event']
                    print(f"{event['label']}: {event['signature']}")
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
except Exception as e:
    print(f"Request failed: {e}")

{
  "hash": "0xfeda0e8f0d6e54112c28d319c0d303c065d1125c9197bd653682f5fcb0a6c81e",
  "nonce": "327",
  "transaction_index": "176",
  "from_address_entity": null,
  "from_address_entity_logo": null,
  "from_address": "0xcb1c1fde09f811b294172696404e88e658659905",
  "from_address_label": "reubenrjs.eth",
  "to_address_entity": "1inch",
  "to_address_entity_logo": "https://entities-logos.s3.us-east-1.amazonaws.com/1inch.png",
  "to_address": "0x111111125421ca6dc452d289314280a0f8842a65",
  "to_address_label": "1inch: AggregationRouterV6",
  "value": "0",
  "gas": "217574",
  "gas_price": "848440939",
  "input": "0x07ed23790000000000000000000000005141b82f5ffda4c6fe1e372978f1c5427640a190000000000000000000000000a0b86991c6218b36c1d19d4a2e9eb0ce3606eb48000000000000000000000000eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee0000000000000000000000005141b82f5ffda4c6fe1e372978f1c5427640a190000000000000000000000000cb1c1fde09f811b294172696404e88e65865990500000000000000000000000000000000000000000000000000000000

In [12]:
import requests
import json
from dotenv import load_dotenv
import os
from datetime import datetime

load_dotenv()

API_KEY = os.getenv("MORALIS_KEY")
if not API_KEY:
    raise ValueError("MORALIS_KEY not found in .env file")

# Wallet address to query
WALLET_ADDRESS = "0xcB1C1FdE09f811B294172696404e88E658659905"

# Build URL for wallet transactions
url = f"https://deep-index.moralis.io/api/v2.2/{WALLET_ADDRESS}/verbose?chain=eth&order=DESC&limit=25"

headers = {
    "Accept": "application/json",
    "X-API-Key": API_KEY
}

try:
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        
        # Print pagination info
        print(f"Page: {data.get('page')}/{data.get('page_size')} transactions per page")
        print(f"Total transactions fetched: {len(data.get('result', []))}\n")
        
        # Extract and display transaction summaries
        if data.get('result'):
            print("=" * 120)
            for idx, tx in enumerate(data['result'], 1):
                print(f"\n{idx}. Transaction Hash: {tx['hash']}")
                print(f"   From: {tx['from_address_label'] or tx['from_address']}")
                print(f"   To: {tx['to_address_label'] or tx['to_address']}")
                print(f"   Value: {float(tx['value']) / 1e18:.4f} ETH")
                print(f"   Block: {tx['block_number']}")
                print(f"   Time: {tx['block_timestamp']}")
                print(f"   Status: {'✓ Success' if tx['receipt_status'] == '1' else '✗ Failed'}")
                print(f"   Gas Used: {tx['receipt_gas_used']}")
                print(f"   Fee: {float(tx['transaction_fee']):.6f} ETH")
                
                # Show decoded function call if available
                if tx.get('decoded_call'):
                    call = tx['decoded_call']
                    print(f"   Function: {call['label']} ({call['signature']})")
                
                # Show events
                if tx.get('logs'):
                    decoded_events = [log for log in tx['logs'] if log.get('decoded_event')]
                    if decoded_events:
                        print(f"   Events: {len(decoded_events)}")
                        for event in decoded_events[:2]:  # Show first 2 events
                            print(f"     - {event['decoded_event']['label']}")
            
            print("\n" + "=" * 120)
            
            # Handle pagination
            if data.get('cursor'):
                print(f"\nMore transactions available. Use cursor: {data['cursor'][:50]}...")
                print("To fetch next page, add: &cursor={cursor_value}")
        else:
            print("No transactions found")
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        
except Exception as e:
    print(f"Request failed: {e}")

Page: 1/25 transactions per page
Total transactions fetched: 25


1. Transaction Hash: 0xf53ef12d0b1604acee0a30b27e025980a3370e51cf087581ae8adc3800f3d0ae
   From: reubenrjs.eth
   To: 0x4fef9d741011476750a243ac70b9789a63dd47df
   Value: 0.0010 ETH
   Block: 23583751
   Time: 2025-10-15T14:41:23.000Z
   Status: ✓ Success
   Gas Used: 71112
   Fee: 0.000176 ETH
   Function: deposit (deposit(address,address))

2. Transaction Hash: 0x2672ff7ec16848ea46b2269ea5a138a67215fb62550e41a39e41c1098b2a871b
   From: reubenrjs.eth
   To: OpenSea: Seaport
   Value: 0.0019 ETH
   Block: 23371336
   Time: 2025-09-15T22:03:35.000Z
   Status: ✓ Success
   Gas Used: 194278
   Fee: 0.000291 ETH
   Function: fulfillBasicOrder_efficient_6GL6yc (fulfillBasicOrder_efficient_6GL6yc(tuple))
   Events: 2
     - OrderFulfilled
     - Transfer

3. Transaction Hash: 0xe630b9487f89a8256265c0565701a75e11e52d8a942a26f4714741659a90b654
   From: reubenrjs.eth
   To: Socket: Socket Gateway
   Value: 0.0160 ETH
   Block: 22